# Charge Time Machine Learning Algorithm Generation

## Splitting the data

In [1]:
import pandas as pd
import datetime as td
import  numpy as np
import datetime 
import datetime as dt
df = pd.read_csv('data/chargeML.csv')
df=df._convert(numeric=True)
df['date2'] =  pd.to_datetime(df['date2'])
#df['date2'] =  pd.to_datetime(df['date2'])
df["dayInMonth"] =df['date2'].dt.day
inData=dataSet=df

In [2]:
df

,date2,ActualGenerationMW,ForecastWind,ActualWind,ActualDemand,ker_temp,dub_temp,dub_wddir,don_wddir,dub_wdsp,...,dayOfWeek,dayOfYear,sin_hour,sin_day,cos_hour,cos_day,sin_wind,cos_wind,totalImbalance,dayInMonth
0,2019-04-01 00:00:00,2681.0,273.0,555.0,3430.0,6.3,8.4,90.0,150.0,4.0,...,0,91,0.000000,0.999991,0.000000,0.999991,0.598460,-0.801153,-749.0,1
1,2019-04-01 01:00:00,2696.0,241.0,470.0,3281.0,4.6,8.5,110.0,150.0,4.0,...,0,91,0.258819,0.999991,0.258819,0.999991,0.836379,-0.548152,-585.0,1
2,2019-04-01 02:00:00,2834.0,222.0,372.0,3149.0,4.8,8.7,130.0,140.0,3.0,...,0,91,0.500000,0.999991,0.500000,0.999991,0.973418,-0.229036,-315.0,1
3,2019-04-01 03:00:00,2886.0,225.0,342.0,3073.0,3.7,8.3,190.0,140.0,4.0,...,0,91,0.707107,0.999991,0.707107,0.999991,0.685060,0.728487,-187.0,1
4,2019-04-01 04:00:00,2891.0,253.0,342.0,3024.0,3.3,7.7,320.0,150.0,2.0,...,0,91,0.866025,0.999991,0.866025,0.999991,-0.998401,0.056524,-133.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26240,2022-03-31 19:00:00,5542.0,1949.0,1891.0,5291.0,6.8,5.9,330.0,360.0,6.0,...,3,90,-0.965926,0.999769,-0.965926,0.999769,-0.993049,-0.117705,251.0,31
26241,2022-03-31 20:00:00,5506.0,1781.0,1434.0,5254.0,6.6,4.0,320.0,360.0,5.0,...,3,90,-0.866025,0.999769,-0.866025,0.999769,-0.998401,0.056524,252.0,31
26242,2022-03-31 21:00:00,5514.0,1716.0,1379.0,5369.0,6.6,3.2,320.0,360.0,5.0,...,3,90,-0.707107,0.999769,-0.707107,0.999769,-0.998401,0.056524,145.0,31
26243,2022-03-31 22:00:00,5042.0,1635.0,1422.0,4907.0,6.3,3.0,330.0,10.0,5.0,...,3,90,-0.500000,0.999769,-0.500000,0.999769,-0.993049,-0.117705,135.0,31


In [47]:
type(df['date2'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [58]:
from sklearn.model_selection import train_test_split
import numpy as np

splitOption = 1 # split per day of the month

#cutOffTestDate=2022.01.01
cutOffTestDate = datetime.datetime(2022, 1, 1)
testSet = dataSet.loc[(inData.date2 > cutOffTestDate), :]
mainSet = dataSet.loc[(inData.date2 <= cutOffTestDate), :]


In [64]:
trainSet = dataSet.loc[(dataSet.dayInMonth > 0) & (dataSet.dayInMonth < 23), :]
validSet = dataSet.loc[(dataSet.dayInMonth >= 23) & (inData.date2 <= cutOffTestDate), :]

In [67]:
'''
if (splitOption == 0):  # Standard SkLearn train test split, usually not good for time series

elif (splitOption == 1):  # split before / after day of the month
  dataSet["dayInMonth"] = 0

  def setDayInMonth(row):
      row["dayInMonth"] = row.date.day
      return row

  dataSet = dataSet.apply(setDayInMonth, axis=1)

  trainSet = dataSet.loc[(dataSet.dayInMonth > 0) & (dataSet.dayInMonth < 23), :]
  validSet = dataSet.loc[(dataSet.dayInMonth >= 23) & (inData.date <= cutOffTestDate), :]
elif (splitOption == 2):  # split per period
  trainSet = dataSet.loc[(inData.date <= cutOffValidationDate), :]
  validSet = dataSet.loc[(inData.date > cutOffValidationDate), :]  
  '''

'\nif (splitOption == 0):  # Standard SkLearn train test split, usually not good for time series\n\nelif (splitOption == 1):  # split before / after day of the month\n  dataSet["dayInMonth"] = 0\n\n  def setDayInMonth(row):\n      row["dayInMonth"] = row.date.day\n      return row\n\n  dataSet = dataSet.apply(setDayInMonth, axis=1)\n\n  trainSet = dataSet.loc[(dataSet.dayInMonth > 0) & (dataSet.dayInMonth < 23), :]\n  validSet = dataSet.loc[(dataSet.dayInMonth >= 23) & (inData.date <= cutOffTestDate), :]\nelif (splitOption == 2):  # split per period\n  trainSet = dataSet.loc[(inData.date <= cutOffValidationDate), :]\n  validSet = dataSet.loc[(inData.date > cutOffValidationDate), :]  \n  '

In [70]:
y_train = trainSet.ActualWind
y_valid = validSet.ActualWind
y_test = testSet.ActualWind

In [72]:
X_train = trainSet
X_valid = validSet 

## Random forest generation

In [79]:
y_test.dtypes

dtype('float64')

In [78]:
X_train.dtypes

date2                 datetime64[ns]
ActualGenerationMW           float64
ForecastWind                 float64
ActualWind                   float64
ActualDemand                 float64
ker_temp                     float64
dub_temp                     float64
dub_wddir                    float64
don_wddir                    float64
dub_wdsp                     float64
may_wdsp                     float64
dub_msl                      float64
hour                           int64
dayOfWeek                      int64
dayOfYear                      int64
sin_hour                     float64
sin_day                      float64
cos_hour                     float64
cos_day                      float64
sin_wind                     float64
cos_wind                     float64
totalImbalance               float64
dayInMonth                     int64
dtype: object

In [81]:
X_train1 = X_train[['ActualWind','dayOfWeek','ActualDemand' ]]

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train1,y_train)

y_pred=clf.predict(X_test)

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);